In [1]:
import os
import numpy as np
import pandas as pd
np.set_printoptions(suppress=True)

In [2]:
def mkdir(path):
	folder = os.path.exists(path)
 
	if not folder:                   #判断是否存在文件夹如果不存在则创建为文件夹
		os.makedirs(path)            #makedirs 创建文件时如果路径不存在会创建这个路径
	print('本次分析结果均储存在 %s\\%s' %(os.getcwd(),path))
    
def score_distribute(data,var='age'):
    score_table=pd.DataFrame()
    distribute_cnt=data.groupby(var)['apply_no'].count()
    ratio = distribute_cnt/data['apply_no'].count()
    score_table['count']=distribute_cnt
    score_table['ratio']=ratio
    score_table.reset_index(inplace=True)
    
    if sum(data[var].isnull()) != 0:
        data_nan=data[data[var].isnull()].copy()
        nan_table=pd.DataFrame()
        nan_table[var]=pd.Series('NaN')
        nan_table['count'] = data_nan.shape[0]
        nan_table['ratio'] = data_nan.shape[0]/data['apply_no'].count()
   
    total_table=pd.DataFrame()
    total_table[var]=pd.Series('总计')
    total_table['count']=data['apply_no'].count()
    total_table['ratio']=data['apply_no'].count()/data['apply_no'].count()
    
    if sum(data[var].isnull()) != 0:
        table_all=pd.concat([score_table,nan_table,total_table],axis=0)
    else:
        table_all=pd.concat([score_table,total_table],axis=0)
    return table_all

def Out_Excel_score(file_name,sheetname='score_distribute_combine',col_start=2,row_start=7,rgap=2):
    col=col_start;row=row_start
    writer = pd.ExcelWriter(file_name,engine='xlsxwriter')
    null=pd.DataFrame()
    null.to_excel(writer,sheet_name = sheetname, startcol = col, startrow = row)
    workbook = writer.book
    worksheet = writer.sheets[sheetname]
    
    worksheet.write(row,col,'combine');row=row+2
    for i in range(len(score_list)):
        #worksheet.write(row,col,score_list[i]);row=row+1
        score_distribute_combine[i].to_excel(writer,sheet_name = sheetname, startcol = col, startrow = row,index=False)
        row=row+score_distribute_combine[i].shape[0]+3
    writer.close()

# 1.数据导入

In [3]:
# 入参文档位置
recan_file_path = './20220402/2.csv'

# 本次分析内容（有限/无限）
con = '有限' #分析什么的时候就放入什么数据

# 输出文件夹名称
output_dir_name = '分析结果202104'

# 清洗后原始数据储存位置
data_cleaned_path = '入参-converted-NBTSPC034.csv' #文件名

# 分析结果文档
stat_output_path = 'score_distribute_combine_limit.xlsx'

In [4]:
mkdir(output_dir_name)
data_cleaned_path= os.path.join('./'+output_dir_name, data_cleaned_path)
stat_output_path= os.path.join('./'+output_dir_name, stat_output_path)

## 导入数据
# data_rucan= pd.read_csv('C:\\Users\\fanyihao\\Desktop\\20220118\\萨摩耶入参.txt')
# data_shenpi= pd.read_csv('C:\\Users\\fanyihao\\Desktop\\20220118\\萨摩耶审批.txt')
data_rucan= pd.read_csv(recan_file_path)
# data_shenpi= pd.read_csv('C:\\Users\\fanyihao\\Desktop\\20220118\\萨摩耶审批.txt')
data_rucan.columns=['apply_no', 'product_id', 'api_type', 'index_name', 'score']

# 有限
data_rucan_limit = data_rucan[data_rucan.product_id=='NBTSPC034']
# 无限
data_rucan_unlimit = data_rucan[data_rucan.product_id=='NBTSPC035']
# data_rucan.head(1)

print('本次分析全量共计'+str(data_rucan['apply_no'].nunique())+'笔，其中：')
print('有限担保笔数共计'+str(data_rucan_limit['apply_no'].nunique())+'笔。')
print('无限担保笔数共计'+str(data_rucan_unlimit['apply_no'].nunique())+'笔。')

data_rucan= data_rucan_limit if con=='有限' else data_rucan_unlimit
print()
print('当前分析板块为'+con+', product_id为'+str(data_rucan['product_id'].unique()[0])+'。')
print()
print('开始处理原始数据表...')
#limit中用“dt_result.IN_按身份证号查询近1个月在非银机构申请机构数or3个月”;
#unlimit中用“wbfqzdt_result.IN_按身份证号查询近1个月在非银机构申请机构数or3个月”
#limit中有“rhsxsp_result.IN_24个月内贷记卡累计逾期次数”
#unlimit中没有rhsxsp_result.IN_24个月内贷记卡累计逾期次数字段“

data_rucan_pivot = data_rucan.pivot(index='apply_no',columns='index_name',values='score')

# # IN_是否有准贷记卡记录/IN_是否有贷记卡记录/IN_是否有贷款记录 任一标签为-999999，则 [是否有信贷记录] 标签为-999999
# # IN_是否有准贷记卡记录/IN_是否有贷记卡记录/IN_是否有贷款记录 和大于0，则 [是否有信贷记录]=1， 否则为0
# data_rucan_pivot_v1['是否有信贷记录']=np.where((data_rucan_pivot_v1['final_result.IN_是否有准贷记卡记录']+
#                                         data_rucan_pivot_v1['final_result.IN_是否有贷记卡记录']+
#                                         data_rucan_pivot_v1['final_result.IN_是否有贷款记录'])>0,1,
#                                         np.where((data_rucan_pivot_v1['final_result.IN_是否有准贷记卡记录']==-999999)
#                                                  |(data_rucan_pivot_v1['final_result.IN_是否有贷记卡记录']==-999999)
#                                                  |(data_rucan_pivot_v1['final_result.IN_是否有贷款记录']==-999999),-999999,0))

# # 没有信用卡记录为空，没有满足状态的为0
# data_rucan_pivot_v1['final_result.IN_当前已用额度大于授信额度的信用卡账户数']=np.where(data_rucan_pivot_v1['final_result.IN_是否有贷记卡记录']<1,-999999,
#                                                              np.where((data_rucan_pivot_v1['final_result.IN_是否有贷记卡记录']==1)
#                                                                       &(data_rucan_pivot_v1['final_result.IN_当前已用额度大于授信额度的信用卡账户数']<0),0,
#                                                                         data_rucan_pivot_v1['final_result.IN_当前已用额度大于授信额度的信用卡账户数']))
# # 没有贷款记录,没有房贷为0
# data_rucan_pivot_v1['final_result.IN_所有机构发放的房贷笔数']=np.where(data_rucan_pivot_v1['final_result.IN_是否有贷款记录']<1,-999999,
#                                                             np.where((data_rucan_pivot_v1['final_result.IN_是否有贷款记录']==1)
#                                                                      &(data_rucan_pivot_v1['final_result.IN_所有机构发放的房贷笔数']<0),0,
#                                                                        data_rucan_pivot_v1['final_result.IN_所有机构发放的房贷笔数']))
# # 0和空值为一档
# data_rucan_pivot_v1['final_result.IN_贷款近60个月逾期最大期数']=np.where(data_rucan_pivot_v1['final_result.IN_贷款近60个月逾期最大期数']<0,0,
#                                                                        data_rucan_pivot_v1['final_result.IN_贷款近60个月逾期最大期数'])                           

# # 没有贷记卡为空
# data_rucan_pivot_v1['final_result.IN_近180天有还款贷记卡账户数总额度占总共享额度比例']=np.where(data_rucan_pivot_v1['final_result.IN_是否有贷记卡记录']<1,-999999,
#                                                                           np.where((data_rucan_pivot_v1['final_result.IN_是否有贷记卡记录']==1)
#                                                                                    &(data_rucan_pivot_v1['final_result.IN_近180天有还款贷记卡账户数总额度占总共享额度比例']<0),0,
#                                                                                      data_rucan_pivot_v1['final_result.IN_近180天有还款贷记卡账户数总额度占总共享额度比例']))
# # 没有贷记卡记录
# data_rucan_pivot_v1['final_result.IN_近24个月内发生逾期信用卡合计月份数']=np.where(data_rucan_pivot_v1['final_result.IN_是否有贷记卡记录']<1,-999999,
#                                                                    np.where((data_rucan_pivot_v1['final_result.IN_是否有贷记卡记录']==1)
#                                                                             &(data_rucan_pivot_v1['final_result.IN_近24个月内发生逾期信用卡合计月份数']<0),0,
#                                                                               data_rucan_pivot_v1['final_result.IN_近24个月内发生逾期信用卡合计月份数']))
# # 没有贷款记录为空
# data_rucan_pivot_v1['final_result.IN_额度小于等于5千贷款笔数占比']=np.where(data_rucan_pivot_v1['final_result.IN_是否有贷款记录']<1,-999999,
#                                                               data_rucan_pivot_v1['final_result.IN_额度小于等于5千贷款笔数占比']) 


# # 没有信贷记录或没有逾期记录为空
# # !!!!!!!!!! 逾期次数 交叉
# # X016	当前贷款逾期期数（贷款类） | X017	当前贷记卡逾期期数（所有贷记卡）
# data_rucan_pivot_v1['final_result.IN_信贷最后一次逾期距今月数']=np.where(data_rucan_pivot_v1['是否有信贷记录']<1,-999999,
#                                                              data_rucan_pivot_v1['final_result.IN_信贷最后一次逾期距今月数'])
# # 没有未结清贷款（分母）为空
# # !!!!!!!!!! 未结清贷款数 交叉        
# # X009	贷款未结清笔数 
# # X084	未结清贷款笔数

# data_rucan_pivot_v1['final_result.IN_未结清消费金融贷款笔数占比']=np.where(data_rucan_pivot_v1['final_result.IN_是否有贷款记录']<1,-999999,
#                                                               data_rucan_pivot_v1['final_result.IN_未结清消费金融贷款笔数占比']) # 分母限制BD01

# # 近12月查询次数	没有任何查询记录为空，没有12个月内的查询为0
# # 最近24个月担保资格审查次数_除去本机构	没有查询记录为空
# # -------------- 查询次数

# # 最近一次信用卡审批的时间距今时长_除去本机构	没有信用卡查询记录为空
# # -------------- 信用卡查询次数

# # 平均月缴存额	没有公积金为空，正常计算0值
# # -------------- 是否有公积金记录

data_rucan_pivot.to_csv(data_cleaned_path)  
print()
print('开始统计分析...')

data_rucan_pivot_v1 = pd.read_csv(data_cleaned_path) #文件名

# pd.crosstab(data_rucan_pivot_v1['final_result.IN_近180天有还款贷记卡账户数总额度占总共享额度比例'],data_rucan_pivot_v1['final_result.IN_是否有贷记卡记录'])

data_rucan_pivot_v1['年龄'] = pd.cut(data_rucan_pivot_v1['access_result.IN_年龄'].astype(float),
                                    bins=[22,30,35,40,45,50,np.inf],right=False)

data_rucan_pivot_v1['近12个月内查询次数']=  pd.cut(data_rucan_pivot_v1['final_result.IN_近12个月内查询次数'].astype(float),
                                                 bins=[-999999,0,4,8,12,24,np.inf],right=False)

data_rucan_pivot_v1['未结清消费金融贷款笔数占比']=  pd.cut(data_rucan_pivot_v1['final_result.IN_未结清消费金融贷款笔数占比'].astype(float),
                                                 bins=[-999999,0,0.2,0.5,np.inf],right=False)


data_rucan_pivot_v1['信贷最后一次逾期距今月数']=  pd.cut(data_rucan_pivot_v1['final_result.IN_信贷最后一次逾期距今月数'].astype(float),
                                                 bins=[-999999,0,1,9,24,np.inf],right=False)

data_rucan_pivot_v1['公积金平均月缴存额'] = pd.cut(data_rucan_pivot_v1['final_result.IN_公积金平均月缴存额'].astype(float),
                                                 bins=[-999999,0,300,1000,np.inf],right=False)

data_rucan_pivot_v1['正常超额使用的信用卡张数'] = pd.cut(data_rucan_pivot_v1['final_result.IN_当前已用额度大于授信额度的信用卡账户数'].astype(float),
                                                 bins=[-999999,0,1,2,np.inf],right=False)

data_rucan_pivot_v1['微额(<=5k)贷款笔数占比']=  pd.cut(data_rucan_pivot_v1['final_result.IN_额度小于等于5千贷款笔数占比'].astype(float),
                                                 bins=[-999999,0,0.1,0.5,1,np.inf],right=False)

data_rucan_pivot_v1['最近一次信用卡审批的时间距今时长_除去本机构'] = pd.cut(data_rucan_pivot_v1['final_result.IN_最近一次信用卡审批距离报告日天数'].astype(float),
                                                 bins=[-999999,0,90,540,np.inf],right=False)

data_rucan_pivot_v1['房贷笔数'] = pd.cut(data_rucan_pivot_v1['final_result.IN_所有机构发放的房贷笔数'].astype(float),
                                                 bins=[-999999,0,1,np.inf],right=False)

data_rucan_pivot_v1['贷记卡近180天有还款账户数总额度占总共享额度比例'] = pd.cut(data_rucan_pivot_v1['final_result.IN_近180天有还款贷记卡账户数总额度占总共享额度比例'].astype(float),
                                                 bins=[-999999,0,0.5,1,np.inf],right=False)

data_rucan_pivot_v1['近24个月内发生逾期信用卡合计月份数'] = pd.cut(data_rucan_pivot_v1['final_result.IN_近24个月内发生逾期信用卡合计月份数'].astype(float),
                                                 bins=[-999999,0,1,2,np.inf],right=False)

data_rucan_pivot_v1['最近24个月担保资格审查次数_除去本机构'] = pd.cut(data_rucan_pivot_v1['final_result.IN_最近24个月内的担保资格查询次数'].astype(float),
                                                 bins=[-999999,0,1,np.inf],right=False)

data_rucan_pivot_v1['贷款逾期最长逾期月数'] = pd.cut(data_rucan_pivot_v1['final_result.IN_贷款近60个月逾期最大期数'].astype(float),
                                                 bins=[-999999,0,1,np.inf],right=False)

data_rucan_pivot_v1['百行反欺诈评分']=  pd.cut(data_rucan_pivot_v1['bhsxsp_result.IN_百行反欺诈评分'].astype(float),
                                                 bins=[-1,0,200,400,600,800,825,1000,np.inf],right=False)

data_rucan_pivot_v1['贷记卡当前逾期期数'] = pd.cut(data_rucan_pivot_v1['rhsxsp_result.IN_当前贷记卡逾期期数'].astype(float),
                                                 bins=[ -999999,0,1,2,3,4,5,np.inf],right=False)

data_rucan_pivot_v1['当前贷款逾期期数'] = pd.cut(data_rucan_pivot_v1['rhsxsp_result.IN_当前贷款逾期期数'].astype(float),
                                                 bins=[ -999999,0,1,2,3,4,5,np.inf],right=False)

data_rucan_pivot_v1['近一个月人行查询次数'] = pd.cut(data_rucan_pivot_v1['rhsxsp_result.IN_近一个月人行查询次数'].astype(float),
                                                 bins=[ -999999,0,1,3, 5,7,9,11,14,17,20,np.inf],right=False)

data_rucan_pivot_v1['近3个月人行征信查询次数'] = pd.cut(data_rucan_pivot_v1['rhsxsp_result.IN_近3个月人行查询次数'].astype(float),
                                                 bins=[ -999999,0,1,3, 5,7,9,11,14,17,20,np.inf],right=False)

data_rucan_pivot_v1['近6个月人行征信查询次数'] = pd.cut(data_rucan_pivot_v1['rhsxsp_result.IN_近6个月人行查询次数'].astype(float),
                                                 bins=[ -999999,0,1,3, 5,7,9,11,14,17,20,np.inf],right=False)


if con=='有限':
    #limit用
    data_rucan_pivot_v1['百融多头1个月']=  pd.cut(data_rucan_pivot_v1['dt_result.IN_按身份证号查询近1个月在非银机构申请机构数'].astype(float),
                                                     bins=[-999999,0,3,5,10,15,np.inf],right=False)
    #limit用
    data_rucan_pivot_v1['百融多头3个月']=  pd.cut(data_rucan_pivot_v1['dt_result.IN_按身份证号查询近3个月在非银机构申请机构数'].astype(float),
                                                     bins=[-999999,0,5,10,15,20,25,np.inf],right=False)
    
    data_rucan_pivot_v1['近24个月贷记卡累计逾期次数'] = pd.cut(data_rucan_pivot_v1['rhsxsp_result.IN_24个月内贷记卡累计逾期次数'].astype(float),
                                                 bins=[ -999999,0,1,2,3,4,5,6,7,8,9,np.inf],right=False)

    # unlimit模式中无  连续=最高
    data_rucan_pivot_v1['近12个月贷记卡连续（最高）逾期期数'] = pd.cut(data_rucan_pivot_v1['rhsxsp_result.IN_近12个月内贷记卡最高逾期期数'].astype(float),
                                                     bins=[ -999999,0,1,2,3,4,5,6,np.inf],right=False)

    # unlimit模式中无 
    data_rucan_pivot_v1['近3个月贷记卡连续（最高）逾期期数'] = pd.cut(data_rucan_pivot_v1['rhsxsp_result.IN_近3个月内贷记卡最高逾期期数'].astype(float),
                                                     bins=[ -999999,0,1,2,3,np.inf],right=False)
    
    # unlimit模式中无 
    data_rucan_pivot_v1['近24个月贷款累计逾期次数'] = pd.cut(data_rucan_pivot_v1['rhsxsp_result.IN_24个月内贷款累计逾期次数'].astype(float),
                                                     bins=[ -999999,0,1,2,3,4,5,6,7,8,9,np.inf],right=False)
     # unlimit模式中无 
    data_rucan_pivot_v1['近3个月贷款连续逾期期数'] = pd.cut(data_rucan_pivot_v1['rhsxsp_result.IN_近3个月内贷款连续逾期月份数'].astype(float),
                                                     bins=[ -999999,0,1,2,3,np.inf],right=False)

if con=='无限':
    #unlimit用
    data_rucan_pivot_v1['微博多头1个月']=  pd.cut(data_rucan_pivot_v1['wbfqzdt_result.IN_按身份证号查询近1个月在非银机构申请机构数'].astype(float),
                                                     bins=[-999999,0,3,5,10,15,np.inf],right=False)
    #unlimit用
    data_rucan_pivot_v1['微博多头3个月']=  pd.cut(data_rucan_pivot_v1['wbfqzdt_result.IN_按身份证号查询近3个月在非银机构申请机构数'].astype(float),
                                                     bins=[-999999,0,5,10,15,20,25,np.inf],right=False)

## 输入待分析的变量（切分分档后的变量名）
#limit 
if con=='有限':
    score_list=['年龄','近12个月内查询次数','未结清消费金融贷款笔数占比','信贷最后一次逾期距今月数','公积金平均月缴存额',
                '正常超额使用的信用卡张数','微额(<=5k)贷款笔数占比','最近一次信用卡审批的时间距今时长_除去本机构',
                '房贷笔数','贷记卡近180天有还款账户数总额度占总共享额度比例','近24个月内发生逾期信用卡合计月份数','最近24个月担保资格审查次数_除去本机构',
                '贷款逾期最长逾期月数', '百融多头1个月','百融多头3个月','百行反欺诈评分',
               '贷记卡当前逾期期数','近24个月贷记卡累计逾期次数','近12个月贷记卡连续逾期期数','近3个月贷记卡连续逾期期数','当前贷款逾期期数',
               '近24个月贷款累计逾期次数','近3个月贷款连续逾期期数','近一个月人行查询次数','近3个月人行征信查询次数','近6个月人行征信查询次数']
#unlimit
if con=='无限':
    score_list=['年龄','近12个月内查询次数','未结清消费金融贷款笔数占比','信贷最后一次逾期距今月数','公积金平均月缴存额',
                '正常超额使用的信用卡张数','微额(<=5k)贷款笔数占比','最近一次信用卡审批的时间距今时长_除去本机构',
                '房贷笔数','贷记卡近180天有还款账户数总额度占总共享额度比例','近24个月内发生逾期信用卡合计月份数','最近24个月担保资格审查次数_除去本机构',
                '贷款逾期最长逾期月数', '微博多头1个月','微博多头3个月','百行反欺诈评分',
               '贷记卡当前逾期期数','近12个月贷记卡连续逾期期数','近3个月贷记卡连续逾期期数','当前贷款逾期期数',
               '近24个月贷款累计逾期次数','近3个月贷款连续逾期期数','近一个月人行查询次数','近3个月人行征信查询次数','近6个月人行征信查询次数']

score_distribute_combine=[]
for i in range(len(score_list)):
    try:
        score_distribute_combine.append(score_distribute(data_rucan_pivot_v1,var=score_list[i]))
    except:
        score_distribute_combine.append(pd.DataFrame(columns=[score_list[i],'count','ratio']))

print()
print('开始输出分析文档...')
Out_Excel_score(stat_output_path)##调整输出文档名称
print()
print('Done！')

本次分析结果均储存在 /Users/haozhang/Desktop/入参数据整理/NBTSH-风险分析CODE/1_SMY\分析结果202104
本次分析全量共计1650笔，其中：
有限担保笔数共计846笔。
无限担保笔数共计804笔。

当前分析板块为有限, product_id为NBTSPC034。

开始处理原始数据表...

开始统计分析...

开始输出分析文档...

Done！
